# ML - NCAA Bracket
Using machine learning to predict the men's and women's NCAA tournament brackets

This will be achieved by:
- Compiling historic NCAA basketball data
- Create functions that can be used from that data to predict results
- Train ML model on the historical data, then validate
- Use model to create a bracket for the 2024 and any other upcoming seasons

## Imports


In [108]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import matplotlib as mpl
from matplotlib.patches import Circle, Rectangle, Arc
import seaborn as sns

from sklearn.metrics import accuracy_score, log_loss
import xgboost as xgb # Model that will be used
from sklearn.model_selection import GroupKFold

plt.style.use("fivethirtyeight")
mypal = plt.rcParams["axes.prop_cycle"].by_key()["color"]  # Grab the color pal

## Data Loading and Understanding

In [83]:
!ls -GFlash archive
# Or -R archive

total 291600
     0 drwxr-xr-x@ 34 nvs  staff   1.1K Mar 25 20:31 ./
     0 drwxr-xr-x@  9 nvs  staff   288B Mar 26 11:05 ../
     8 -rw-rw-r--@  1 nvs  staff   1.4K Mar 15 08:04 2024_tourney_seeds.csv
    24 -rw-rw-r--@  1 nvs  staff   9.0K Mar 15 08:04 Cities.csv
     8 -rw-rw-r--@  1 nvs  staff   1.6K Mar 15 08:04 Conferences.csv
   320 -rw-rw-r--@  1 nvs  staff   159K Mar 15 08:04 MConferenceTourneyGames.csv
  5056 -rw-rw-r--@  1 nvs  staff   2.5M Mar 15 08:04 MGameCities.csv
223800 -rw-rw-r--@  1 nvs  staff   109M Mar 15 08:04 MMasseyOrdinals.csv
   144 -rw-rw-r--@  1 nvs  staff    72K Mar 15 08:04 MNCAATourneyCompactResults.csv
   264 -rw-rw-r--@  1 nvs  staff   128K Mar 15 08:04 MNCAATourneyDetailedResults.csv
    32 -rw-rw-r--@  1 nvs  staff    14K Mar 15 08:04 MNCAATourneySeedRoundSlots.csv
    80 -rw-rw-r--@  1 nvs  staff    37K Mar 15 08:04 MNCAATourneySeeds.csv
    96 -rw-rw-r--@  1 nvs  staff    48K Mar 15 08:04 MNCAATourneySlots.csv
 10688 -rw-rw-r--@  1 nvs  staff   5.2M

In [84]:
DATA_PATH = "archive/"

### Putting data into a new league column (W, M) to tell us if it comes from the women''s or men's league 

In [85]:
df_seeds = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MNCAATourneySeeds.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WNCAATourneySeeds.csv").assign(League="W"),
    ],
).reset_index(drop=True)

df_season_results = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MRegularSeasonCompactResults.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WRegularSeasonCompactResults.csv").assign(League="W"),
    ]
).reset_index(drop=True)

df_tourney_results = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MNCAATourneyCompactResults.csv").assign(League="M"),
        pd.read_csv(DATA_PATH + "WNCAATourneyCompactResults.csv").assign(League="W"),
    ]
).reset_index(drop=True)

In [86]:
df_seeds

,Season,Seed,TeamID,League
0,1985,W01,1207,M
1,1985,W02,1210,M
2,1985,W03,1228,M
3,1985,W04,1260,M
4,1985,W05,1374,M
...,...,...,...,...
4093,2023,Z12,3405,W
4094,2023,Z13,3387,W
4095,2023,Z14,3241,W
4096,2023,Z15,3436,W


# Goal:
Modify and aggregate our data into a format that we can use to train a model

This dataset should have 1 row per tournament game:
- Features about each team in that game
- Who won each game

## Step 1: Create regular season team result data

In [105]:
df_season_results.columns

Index(['Season', 'DayNum', 'WTeamID', 'WScore', 'LTeamID', 'LScore', 'WLoc',
       'NumOT', 'League'],
      dtype='object')

### 2 rows for each game, 1 for the winning team, and 1 for the losing team

In [134]:
df_team_season_results = pd.concat(
    [
        df_season_results[["Season", "League", "WTeamID", "DayNum", "WScore", "LScore"]]
        .assign(GameResult="W")
        .rename(
            columns={"WTeamID": "TeamID", "WScore": "TeamScore", "LScore": "OppScore"}
        ),
        df_season_results[["Season", "League", "LTeamID", "DayNum", "WScore", "LScore"]]
        .assign(GameResult="L")
        .rename(
            columns={"LTeamID": "TeamID", "LScore": "TeamScore", "WScore": "OppScore"}
        ),
    ]
).reset_index(drop=True)

In [136]:
df_team_season_results

,Season,League,TeamID,DayNum,TeamScore,OppScore,GameResult
0,1985,M,1228,20,81,64,W
1,1985,M,1106,25,77,70,W
2,1985,M,1112,25,63,56,W
3,1985,M,1165,25,70,54,W
4,1985,M,1192,25,86,74,W
...,...,...,...,...,...,...,...
634889,2024,W,3396,114,67,76,L
634890,2024,W,3129,114,57,63,L
634891,2024,W,3348,114,59,69,L
634892,2024,W,3236,114,61,70,L


### Create score differential column

In [107]:
# Showing how score diff is calculated
df_team_season_results["TeamScore"] - df_team_season_results["OppScore"]

0         17
1          7
2          7
3         16
4         12
          ..
634889    -9
634890    -6
634891   -10
634892    -9
634893   -15
Length: 634894, dtype: int64

In [138]:
# Inserting score differential column
df_team_season_results["ScoreDiff"] = (
    df_team_season_results["TeamScore"] - df_team_season_results["OppScore"]
)
# Inserting win int column
df_team_season_results["Win"] = (df_team_season_results["GameResult"] == "W").astype(
    "int"
)

In [139]:
df_team_season_results.sample(10)

,Season,League,TeamID,DayNum,TeamScore,OppScore,GameResult,ScoreDiff,Win
497966,2023,M,1138,106,61,85,L,-24,0
301353,2021,W,3425,104,66,49,W,17,1
209026,2003,W,3399,43,69,46,W,23,1
91182,2006,M,1318,96,79,74,W,5,1
517356,2001,W,3306,35,44,75,L,-31,0
198373,2000,W,3376,111,64,51,W,13,1
126440,2013,M,1341,68,60,44,W,16,1
410613,2007,M,1182,21,82,87,L,-5,0
18643,1989,M,1333,110,106,66,W,40,1
498432,2023,M,1160,115,65,84,L,-19,0


### Aggregate individual team games into team total season stats
- Take our newely created df_team_season_results dataframe and create aggregated statistics
- Resulting aggregated dataframe will be 1 row per team/season

In [184]:
# Aggregate the data
team_season_agg = (
    df_team_season_results.groupby(["Season", "TeamID", "League"])
    .agg(
        AvgScoreDiff=("ScoreDiff", "mean"),
        MedianScoreDiff=("ScoreDiff", "median"),
        MinScoreDiff=("ScoreDiff", "min"),
        MaxScoreDiff=("ScoreDiff", "max"),
        Wins=("Win", "sum"),
        Losses=("GameResult", lambda x: (x == "L").sum()),
        WinPercentage=("Win", "mean"),
    )
    .reset_index()
)

In [185]:
team_season_agg.sample(10)

,Season,TeamID,League,AvgScoreDiff,MedianScoreDiff,MinScoreDiff,MaxScoreDiff,Wins,Losses,WinPercentage
12063,2010,3190,W,6.709677,11.0,-39,35,23,8,0.741935
20341,2022,1466,M,-8.115385,-8.5,-32,14,5,21,0.192308
14658,2014,1377,M,-6.259259,-4.0,-37,14,9,18,0.333333
350,1986,1190,M,-1.888889,-3.0,-25,29,11,16,0.407407
13461,2012,3212,W,-3.758621,-3.0,-53,29,14,15,0.482759
9597,2007,1106,M,-6.344828,-5.0,-40,17,9,20,0.310345
12589,2011,1381,M,-4.357143,-7.5,-26,21,10,18,0.357143
22058,2024,3382,W,-11.642857,-11.5,-40,18,4,24,0.142857
12904,2011,3352,W,1.800000,3.5,-29,36,16,14,0.533333
21530,2024,1209,M,-1.814815,-4.0,-32,28,11,16,0.407407


### Create chalk seeds, new column (in seeds) that provides a number 1-16

In [144]:
df_seeds

,Season,Seed,TeamID,League
0,1985,W01,1207,M
1,1985,W02,1210,M
2,1985,W03,1228,M
3,1985,W04,1260,M
4,1985,W05,1374,M
...,...,...,...,...
4093,2023,Z12,3405,W
4094,2023,Z13,3387,W
4095,2023,Z14,3241,W
4096,2023,Z15,3436,W


In [188]:
df_seeds["ChalkSeed"] = (
    df_seeds["Seed"].str.replace("a", "").str.replace("b", "").str[1:].astype("int")
)
# Merging new ChalkSeeds with team_season_agg
team_season_agg = team_season_agg.merge(
    df_seeds, on=["Season", "TeamID", "League"], how="left"
)

In [189]:
df_seeds

,Season,Seed,TeamID,League,ChalkSeed
0,1985,W01,1207,M,1
1,1985,W02,1210,M,2
2,1985,W03,1228,M,3
3,1985,W04,1260,M,4
4,1985,W05,1374,M,5
...,...,...,...,...,...
4093,2023,Z12,3405,W,12
4094,2023,Z13,3387,W,13
4095,2023,Z14,3241,W,14
4096,2023,Z15,3436,W,15


In [196]:
team_season_agg.sample(10)

,Season,TeamID,League,AvgScoreDiff,MedianScoreDiff,MinScoreDiff,MaxScoreDiff,Wins,Losses,WinPercentage,Seed,ChalkSeed
11149,2009,1320,M,4.151515,4.0,-30,26,23,10,0.696970,Z12,12.0
6030,2001,3126,W,-6.296296,-3.0,-63,24,10,17,0.370370,NaN,NaN
5144,2000,1199,M,-5.586207,-3.0,-35,26,12,17,0.413793,NaN,NaN
8484,2005,1343,M,1.037037,2.0,-26,22,14,13,0.518519,NaN,NaN
6482,2002,1279,M,7.785714,6.5,-23,47,18,10,0.642857,Z09,9.0
11339,2009,3156,W,4.468750,6.5,-18,35,19,13,0.593750,NaN,NaN
4547,1999,1231,M,6.375000,3.5,-17,56,22,10,0.687500,Y06,6.0
11388,2009,3205,W,-2.965517,-1.0,-27,15,14,15,0.482759,NaN,NaN
7077,2003,1221,M,11.137931,8.0,-24,43,25,4,0.862069,Y14,14.0
7847,2004,1359,M,-6.125000,-6.0,-22,16,8,16,0.333333,NaN,NaN


In [197]:
df_seeds.shape, team_season_agg.shape

((4098, 5), (22150, 12))

## Step 2: Create tournament results aggregation

In [199]:
df_tourney_results.sample(10)

,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,League
2302,2021,145,1116,72,1331,70,N,0,M
685,1995,144,1438,67,1242,58,N,0,M
495,1992,144,1276,75,1329,72,N,0,M
245,1988,145,1181,63,1396,53,N,0,M
1126,2002,144,1242,73,1228,69,N,0,M
2827,2003,153,3397,66,3181,56,N,0,W
2069,2017,136,1462,76,1268,65,N,0,M
3581,2015,147,3268,58,3397,48,N,0,W
3616,2016,138,3449,65,3335,53,N,0,W
1459,2008,136,1243,80,1425,67,N,0,M


In [200]:
df_tourney_results.shape

(4034, 9)

### 2 rows for each game, 1 for the winning team, and 1 for the losing team (just like before with season)

In [202]:
df_team_tourney_results = pd.concat(
    [
        df_tourney_results[
            ["Season", "League", "WTeamID", "LTeamID", "WScore", "LScore"]
        ]
        .assign(GameResult="W")
        .rename(
            columns={"WTeamID": "TeamID", "LTeamID": "OppTeamID", "Wscore": "TeamScore", "LScore": "OppScore"}
        ),
        df_tourney_results[
            ["Season", "League", "WTeamID", "LTeamID", "WScore", "LScore"]
        ]
        .assign(GameResult="L")
        .rename(
            columns={"LTeamID": "TeamID", "WTeamID": "OppTeamID", "LScore": "TeamScore", "WScore": "OppScore"}
        ),
    ]
).reset_index(drop=True)

df_team_tourney_results["Win"] = (df_team_tourney_results["GameResult"] == "W").astype(
    "int"
)

In [204]:
df_team_tourney_results.shape

(8068, 9)

In [210]:
df_team_tourney_results.sample(5)

,Season,League,TeamID,OppTeamID,WScore,OppScore,GameResult,TeamScore,Win
2273,2021,M,1160,1207,96.0,73,W,NaN,1
5436,2007,M,1460,1338,NaN,79,L,58.0,0
5060,2001,M,1448,1139,NaN,79,L,63.0,0
348,1990,M,1123,1257,62.0,60,W,NaN,1
1649,2011,M,1421,1114,81.0,77,W,NaN,1


## Merge the 2 main df together for model (team season agg and tournament result)
- Merge team season data twice, once for the main team id and the opp id

In [211]:
df_historic_tourney_features = df_team_tourney_results.merge(
    team_season_agg[
        ["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "ChalkSeed"]
    ],
    on=["Season", "League", "TeamID"],
    how="left",
).merge(
    team_season_agg[
        ["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff", "ChalkSeed"]
    ].rename(
        columns={"TeamID": "OppTeamID", "WinPercentage": "OppWinPercentage", "MedianScoreDiff": "OppMedianScoreDiff", "ChalkSeed": "OppChalkSeed"}
    ),
    on=["Season", "League", "OppTeamID"],
)

In [213]:
df_historic_tourney_features.sample(5)

,Season,League,TeamID,OppTeamID,WScore,OppScore,GameResult,TeamScore,Win,WinPercentage,MedianScoreDiff,ChalkSeed,OppWinPercentage,OppMedianScoreDiff,OppChalkSeed
5945,2008,W,3275,3257,NaN,81,L,67.0,0,0.696970,8.0,13.0,0.727273,16.0,4.0
3670,2015,M,1428,1372,57.0,50,W,NaN,1,0.741935,16.0,5.0,0.866667,12.5,12.0
6251,2011,W,3163,3216,75.0,39,W,NaN,1,0.969697,26.0,1.0,0.531250,2.5,16.0
7849,2011,M,1112,1163,NaN,65,L,63.0,0,0.794118,9.0,5.0,0.742857,5.0,3.0
1826,2000,M,1337,1231,77.0,57,W,NaN,1,0.750000,7.5,11.0,0.714286,8.0,6.0


### Additional tourney features
Create features that are the difference between the two given teams:
- Win percentage difference
- Seed difference

In [215]:
df_historic_tourney_features["WinPercentage"] - df_historic_tourney_features["OppWinPercentage"]

0      -0.030303
1      -0.059310
2       0.546616
3       0.062169
4       0.025926
          ...   
8063   -0.223656
8064   -0.071264
8065   -0.320430
8066   -0.062366
8067   -0.120833
Length: 8068, dtype: float64

In [216]:
df_historic_tourney_features["WinPctDiff"] = (
    df_historic_tourney_features["WinPercentage"] - df_historic_tourney_features["OppWinPercentage"]
)
df_historic_tourney_features["ChalkSeedDiff"] = (
    df_historic_tourney_features["ChalkSeed"] - df_historic_tourney_features["OppChalkSeed"]
)
df_historic_tourney_features["H2HMedianScoreDiff"] = (
    df_historic_tourney_features["MedianScoreDiff"] - df_historic_tourney_features["OppMedianScoreDiff"]
)

In [217]:
df_historic_tourney_features.columns

Index(['Season', 'League', 'TeamID', 'OppTeamID', 'WScore', 'OppScore',
       'GameResult', 'TeamScore', 'Win', 'WinPercentage', 'MedianScoreDiff',
       'ChalkSeed', 'OppWinPercentage', 'OppMedianScoreDiff', 'OppChalkSeed',
       'WinPctDiff', 'ChalkSeedDiff', 'H2HMedianScoreDiff'],
      dtype='object')

## Add 538 data
- 538 is a great website to gather additional data for march madness
- The 538 data only goes back to 2016

In [219]:
FiveThirtyEight_scores = pd.concat(
    [
        pd.read_csv("538ratingsMen.csv").assign(
            League="M"
        ),
        pd.read_csv("538ratingsWomen.csv").assign(
            League="W"
        )
    ]
).reset_index(drop=True)

In [225]:
FiveThirtyEight_scores.sample(5)

,Season,TeamID,TeamName,538rating,League
64,2016,1380,Southern,67.96,M
599,2017,3146,Central Arkansas,68.04,W
325,2021,1101,Abilene Christian,77.04,M
585,2017,3320,Northern Iowa,77.10,W
760,2021,3393,Syracuse,85.25,W


### Merge the 538 data onto our dataset

In [227]:
df_historic_tourney_features = df_historic_tourney_features.merge(
    FiveThirtyEight_scores.drop("TeamName", axis=1),
    on=["Season", "League", "TeamID"],
    how="left",
).dropna(subset=["538rating"])
df_historic_tourney_features = df_historic_tourney_features.merge(
    FiveThirtyEight_scores.drop("TeamName", axis=1).rename(
        columns={"TeamID": "OppTeamID"}
    ),
    on=["Season", "League", "OppTeamID"],
    how="left",
    suffixes=("", "Opp"),
)

In [229]:
df_historic_tourney_features.sample(5)

,Season,League,TeamID,OppTeamID,WScore,OppScore,GameResult,TeamScore,Win,WinPercentage,MedianScoreDiff,ChalkSeed,OppWinPercentage,OppMedianScoreDiff,OppChalkSeed,WinPctDiff,ChalkSeedDiff,H2HMedianScoreDiff,538rating,538ratingOpp
1227,2018,W,3110,3417,NaN,71,L,60.0,0,0.812500,9.0,14.0,0.774194,9.0,3.0,0.038306,11.0,0.0,71.55,94.98
1780,2022,M,1411,1242,NaN,83,L,56.0,0,0.600000,2.5,16.0,0.823529,10.0,1.0,-0.223529,15.0,-7.5,71.14,91.27
996,2016,W,3120,3124,NaN,84,L,52.0,0,0.612903,4.0,9.0,0.970588,21.5,1.0,-0.357685,8.0,-17.5,80.17,101.59
78,2016,M,1332,1181,82.0,68,W,NaN,1,0.818182,10.0,1.0,0.696970,8.0,4.0,0.121212,-3.0,2.0,88.00,87.33
1375,2021,W,3163,3219,102.0,59,W,NaN,1,0.960000,32.0,1.0,0.769231,15.5,16.0,0.190769,-15.0,16.5,104.27,66.46


- Create differential feature based on the newly added 538 scores

In [230]:
df_historic_tourney_features["538rating_diff"] = (
    df_historic_tourney_features["538rating"] - df_historic_tourney_features["538ratingOpp"]
)

In [231]:
df_historic_tourney_features.columns

Index(['Season', 'League', 'TeamID', 'OppTeamID', 'WScore', 'OppScore',
       'GameResult', 'TeamScore', 'Win', 'WinPercentage', 'MedianScoreDiff',
       'ChalkSeed', 'OppWinPercentage', 'OppMedianScoreDiff', 'OppChalkSeed',
       'WinPctDiff', 'ChalkSeedDiff', 'H2HMedianScoreDiff', '538rating',
       '538ratingOpp', '538rating_diff'],
      dtype='object')

In [233]:
df_historic_tourney_features[
    ["Season", "TeamID", "538rating", "538ratingOpp", "538rating_diff"]
].sample(5)

,Season,TeamID,538rating,538ratingOpp,538rating_diff
1035,2017,3257,92.24,76.21,16.03
1646,2023,3181,90.13,67.49,22.64
1328,2019,3346,80.50,86.59,-6.09
1122,2017,3332,83.06,111.43,-28.37
1532,2022,3416,87.39,81.32,6.07


# Creating model baseline (where the higher seed always wins)
- Chalk bracket
- Create this baseline to understand if the model is any good

In [235]:
df_historic_tourney_features["ChalkSeed"] > df_historic_tourney_features["OppChalkSeed"]

0       False
1       False
2       False
3       False
4        True
        ...  
1829     True
1830    False
1831     True
1832    False
1833    False
Length: 1834, dtype: bool

In [236]:
df_historic_tourney_features["BaselinePred"] = (
    df_historic_tourney_features["ChalkSeed"] < df_historic_tourney_features["OppChalkSeed"]
)
df_historic_tourney_features.loc[
    df_historic_tourney_features["ChalkSeed"] == df_historic_tourney_features["OppChalkSeed"], "BaselinePred",
] = (
    df_historic_tourney_features["WinPercentage"] > df_historic_tourney_features["OppWinPercentage"]
)

In [241]:
df_historic_tourney_features[
    ["Season", "League", "ChalkSeed", "OppChalkSeed", "BaselinePred"]
].sample(5)

,Season,League,ChalkSeed,OppChalkSeed,BaselinePred
298,2018,M,12.0,5.0,False
892,2023,M,6.0,5.0,False
435,2019,M,7.0,10.0,True
935,2016,W,7.0,10.0,True
1424,2021,W,4.0,12.0,True


# Print out the baseline chalk seed score per season and average accuracy

In [245]:
cv_scores_baseline = []
for season in df_historic_tourney_features["Season"].unique():
    pred = df_historic_tourney_features.query("Season == @season")[
        "BaselinePred"
    ].astype("int")
    y = df_historic_tourney_features.query("Season == @season")["Win"]
    score = accuracy_score(y, pred)
    score_ll = log_loss(y, pred)
    cv_scores_baseline.append(score)
    print(f"Holdout season {season} - Accuracy {score:0.4f} Log Loss {score_ll:0.4f}")

print(f"Baseline accuracy {np.mean(cv_scores_baseline):0.4f}")

Holdout season 2016 - Accuracy 0.6846 Log Loss 11.3676
Holdout season 2017 - Accuracy 0.7769 Log Loss 8.0405
Holdout season 2018 - Accuracy 0.7000 Log Loss 10.8131
Holdout season 2019 - Accuracy 0.7538 Log Loss 8.8723
Holdout season 2021 - Accuracy 0.7519 Log Loss 8.9411
Holdout season 2022 - Accuracy 0.7015 Log Loss 10.7593
Holdout season 2023 - Accuracy 0.7164 Log Loss 10.2213
Baseline accuracy 0.7265


### The baseline prediction chalk seed previously created is 72.7% accurate compared to real history
### Meaning our model should beat 72.7%, or else it isn't a good model

# XGBoost Model
- Goal is to beat 72.7%
- Train using a "leave one season out" validation method
- Train multiple models, holding out one season worth of data, then score the holdout season

In [248]:
# Call on previously created features, leaving some out as the model will be more accurate 
FEATURES = [
    #     "WinPercentage",
    #     "MedianScoreDiff",
    #     "ChalkSeed",
    #     "OppWinPercentage",
    #     "OppMedianScoreDiff",
    #     "OppChalkSeed",
    "WinPctDiff",
    "ChalkSeedDiff",
    #     "538rating",
    #     "538ratingOpp",
    "538rating_diff",
]
TARGET = "Win"

X = df_historic_tourney_features[FEATURES]
y = df_historic_tourney_features[TARGET]
groups = df_historic_tourney_features["Season"]
seasons = df_historic_tourney_features["Season"].unique()

# Setting up cross validation (one season held out)
gkf = GroupKFold(n_splits=df_historic_tourney_features["Season"].nunique())
cv_results = []
models = []

# Looping through each season
season_idx = 0
for train_index, test_index in gkf.split(X, y, groups):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    # Preparing the model
    model = xgb.XGBRegressor(
        eval_metric="logloss",
        n_estimators=1_000,
        learning_rate=0.001,
    )
    holdout_season = seasons[season_idx]
    print(f"Holdout Season: {holdout_season}")
    
    # Training the model
    model.fit(X_train, y_train, eval_set=[(X_test, y_test)], verbose=100)

    # Predicting the test set
    y_pred = model.predict(X_test)
    score_11 = log_loss(y_test, y_pred)
    y_pred = y_pred > 0.5

    # Evaluating the model
    accuracy = accuracy_score(y_test, y_pred)
    cv_results.append(accuracy)
    season_idx += 1
    print(f"Season {holdout_season}: {accuracy} {score_11}")
    models.append(model)

# Print average accuracy across all folds
print("Average CV Accuracy:", np.mean(cv_results))

Holdout Season: 2016
[0]	validation_0-logloss:0.69287
[100]	validation_0-logloss:0.66825
[200]	validation_0-logloss:0.64908
[300]	validation_0-logloss:0.63335
[400]	validation_0-logloss:0.61986
[500]	validation_0-logloss:0.60935
[600]	validation_0-logloss:0.59996
[700]	validation_0-logloss:0.59197
[800]	validation_0-logloss:0.58747
[900]	validation_0-logloss:0.58413
[999]	validation_0-logloss:0.58217
Season 2016: 0.7014925373134329 0.5821676845927166
Holdout Season: 2017
[0]	validation_0-logloss:0.69285
[100]	validation_0-logloss:0.66734
[200]	validation_0-logloss:0.64666
[300]	validation_0-logloss:0.62948
[400]	validation_0-logloss:0.61425
[500]	validation_0-logloss:0.60199
[600]	validation_0-logloss:0.59198
[700]	validation_0-logloss:0.58352
[800]	validation_0-logloss:0.57615
[900]	validation_0-logloss:0.56981
[999]	validation_0-logloss:0.56473
Season 2017: 0.7089552238805971 0.5647296910413034
Holdout Season: 2018
[0]	validation_0-logloss:0.69275
[100]	validation_0-logloss:0.65695
[

## The average accuracy came out to 73.4%, compared to the 72.7% baseline

# Predict on test set 
- Now that the trained models are created, can use them on future data

In [250]:
TEST_SEASON = 2024

seeds_2024 = pd.read_csv(DATA_PATH + "2024_tourney_seeds.csv")

seeds_2024["ChalkSeed"] = (
    seeds_2024["Seed"].str.replace("a", "").str.replace("b", "").str[1:].astype("int")
)

In [251]:
seeds_2024.head()

,Tournament,Seed,TeamID,ChalkSeed
0,M,W01,1345,1
1,M,W02,1266,2
2,M,W03,1243,3
3,M,W04,1397,4
4,M,W05,1181,5


## Tourney pairs
- Don't know which teams will play each other in later rounds, need to create a tourny_pairs dataframe
- Dataframe will have all possible combinations of games

In [257]:
tourney_pairs = (
    seeds_2024.merge(seeds_2024, on=["Tournament"], suffixes=("", "Opp"))
    .assign(Season=TEST_SEASON)
    .query("TeamID != TeamIDOpp")
    .rename(columns={"Tournament": "League"})
)
tourney_pairs = (
    tourney_pairs.merge(
        team_season_agg[["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff"]],
        on=["Season", "League", "TeamID"],
        how="left",
    ).merge(
        team_season_agg[["Season", "League", "TeamID", "WinPercentage", "MedianScoreDiff"]].rename(
            columns={"TeamID": "TeamIDOpp", "WinPercentage": "OppWinPercentage", "MedianScoreDiff": "OppMedianScoreDiff",}
        ),
        on=["Season", "League", "TeamIDOpp"],
    ).reset_index(drop=True)
)
tourney_pairs["OppChalkSeed"] = (
    tourney_pairs["SeedOpp"]
    .str.replace("a", "")
    .str.replace("b", "")
    .str[1:]
    .astype("int")
)

## Add features to 2024

In [260]:
tourney_pairs = tourney_pairs.merge(
    FiveThirtyEight_scores.drop("TeamName", axis=1),
    on=["Season", "League", "TeamID"],
    how="left",
)
tourney_pairs = tourney_pairs.merge(
    FiveThirtyEight_scores.drop("TeamName", axis=1).rename(
        columns={"TeamID": "TeamIDOpp"}
    ),
    on=["Season", "League", "TeamIDOpp"],
    how="left",
    suffixes=("", "Opp"),
)
# Diff features
tourney_pairs["538rating_diff"] = (tourney_pairs["538rating"] - tourney_pairs["538ratingOpp"])
tourney_pairs["BaselinePred"] = (tourney_pairs["ChalkSeed"] < tourney_pairs["OppChalkSeed"])
tourney_pairs.loc[tourney_pairs["ChalkSeed"] == tourney_pairs["OppChalkSeed"], "BaselinePred",] = (
    tourney_pairs["WinPercentage"] > tourney_pairs["OppWinPercentage"]
)
tourney_pairs["WinPctDiff"] = (
    tourney_pairs["WinPercentage"] - tourney_pairs["OppWinPercentage"]
)
tourney_pairs["ChalkSeedDiff"] = (
    tourney_pairs["ChalkSeed"] - tourney_pairs["OppChalkSeed"]
)
tourney_pairs["MedianScoreDiffDiff"] = (
    tourney_pairs["MedianScoreDiff"] - tourney_pairs["OppMedianScoreDiff"]
)

In [261]:
tourney_pairs.head()

,League,Seed,TeamID,ChalkSeed,SeedOpp,TeamIDOpp,ChalkSeedOpp,Season,WinPercentage,MedianScoreDiff,OppWinPercentage,OppMedianScoreDiff,OppChalkSeed,538rating,538ratingOpp,538rating_diff,BaselinePred,WinPctDiff,ChalkSeedDiff,MedianScoreDiffDiff
0,M,W01,1345,1,W02,1266,2,2024,0.892857,11.0,0.785714,12.0,2,NaN,NaN,NaN,True,0.107143,-1,-1.0
1,M,W03,1243,3,W02,1266,2,2024,0.607143,3.5,0.785714,12.0,2,NaN,NaN,NaN,False,-0.178571,1,-8.5
2,M,W04,1397,4,W02,1266,2,2024,0.785714,15.0,0.785714,12.0,2,NaN,NaN,NaN,False,0.000000,2,3.0
3,M,W05,1181,5,W02,1266,2,2024,0.785714,12.0,0.785714,12.0,2,NaN,NaN,NaN,False,0.000000,3,0.0
4,M,W06,1246,6,W02,1266,2,2024,0.714286,10.0,0.785714,12.0,2,NaN,NaN,NaN,False,-0.071429,4,-2.0


## Creating predictions and aggregating
- Loop through each of the models trained previously, predict on the latest tourney seed data

In [262]:
models

[XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.001, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=1000,
              n_jobs=None, num_parallel_tree=None, random_state=None, ...),
 XGBRegressor(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=Fa

In [263]:
for i, model in enumerate(models):
    tourney_pairs[f"pred_model{i}"] = model.predict(tourney_pairs[FEATURES])

### Now see that every prediction model has been applied to tourney_pairs:

In [270]:
tourney_pairs.head(n=3)

,League,Seed,TeamID,ChalkSeed,SeedOpp,TeamIDOpp,ChalkSeedOpp,Season,WinPercentage,MedianScoreDiff,...,MedianScoreDiffDiff,pred_model0,pred_model1,pred_model2,pred_model3,pred_model4,pred_model5,pred_model6,Pred,ID
0,M,W01,1345,1,W02,1266,2,2024,0.892857,11.0,...,-1.0,0.797576,0.804084,0.622097,0.768386,0.812335,0.665195,0.801917,0.753084,2024_1345_1266
1,M,W03,1243,3,W02,1266,2,2024,0.607143,3.5,...,-8.5,0.726541,0.720515,0.622097,0.768386,0.681913,0.712022,0.649030,0.697215,2024_1243_1266
2,M,W04,1397,4,W02,1266,2,2024,0.785714,15.0,...,3.0,0.726600,0.720903,0.622097,0.768386,0.681913,0.712078,0.649030,0.697287,2024_1397_1266


### Create an average prediction model rating for all:

In [267]:
tourney_pairs[[f for f in tourney_pairs.columns if "model" in f]].mean(axis=1)

0       0.753084
1       0.697215
2       0.697287
3       0.697287
4       0.697215
          ...   
8059    0.697215
8060    0.697215
8061    0.697215
8062    0.697215
8063    0.697215
Length: 8064, dtype: float32

### Store that average in a new column:

In [268]:
tourney_pairs["Pred"] = tourney_pairs[[f for f in tourney_pairs.columns if "model" in f]].mean(axis=1)
tourney_pairs["ID"] = (
    tourney_pairs["Season"].astype("str")
    + "_"
    + tourney_pairs["TeamID"].astype("str")
    + "_"
    + tourney_pairs["TeamIDOpp"].astype("str")
)
preds = tourney_pairs.copy()

In [271]:
tourney_pairs.head(n=3)

,League,Seed,TeamID,ChalkSeed,SeedOpp,TeamIDOpp,ChalkSeedOpp,Season,WinPercentage,MedianScoreDiff,...,MedianScoreDiffDiff,pred_model0,pred_model1,pred_model2,pred_model3,pred_model4,pred_model5,pred_model6,Pred,ID
0,M,W01,1345,1,W02,1266,2,2024,0.892857,11.0,...,-1.0,0.797576,0.804084,0.622097,0.768386,0.812335,0.665195,0.801917,0.753084,2024_1345_1266
1,M,W03,1243,3,W02,1266,2,2024,0.607143,3.5,...,-8.5,0.726541,0.720515,0.622097,0.768386,0.681913,0.712022,0.649030,0.697215,2024_1243_1266
2,M,W04,1397,4,W02,1266,2,2024,0.785714,15.0,...,3.0,0.726600,0.720903,0.622097,0.768386,0.681913,0.712078,0.649030,0.697287,2024_1397_1266


# Bracket Simulation
- Now that there are probabilites for every possible combination of possible game in the tournament, need to convert these into bracket form
- This will be achieved by having the highest scored team move on each round

Bracket code reference: https://www.kaggle.com/code/lennarthaupts/simulate-n-brackets

In [274]:
from tqdm import tqdm

# Load and filter data
round_slots = pd.read_csv(DATA_PATH + "MNCAATourneySlots.csv")
round_slots = round_slots[round_slots["Season"] == 2023]
round_slots = round_slots[
    round_slots["Slot"].str.contains("R")
]  # Filter out First Four

seeds = pd.read_csv(DATA_PATH + "2024_tourney_seeds.csv")
seeds_m = seeds[seeds["Tournament"] == "M"]
seeds_w = seeds[seeds["Tournament"] == "W"]

preds["ID"] = preds["ID"].str.split("_")

In [275]:
def prepare_data(seeds, preds):
    # Function preparing the data for the simulation
    seed_dict = seeds.set_index("Seed")["TeamID"].to_dict()
    inverted_seed_dict = {value: key for key, value in seed_dict.items()}
    probas_dict = {}

    for teams, proba in zip(preds["ID"], preds["Pred"]):
        team1, team2 = teams[1], teams[2]

        probas_dict.setdefault(team1, {})[team2] = proba
        probas_dict.setdefault(team2, {})[team1] = 1 - proba

    return seed_dict, inverted_seed_dict, probas_dict


def simulate(round_slots, seeds, inverted_seeds, probas, sim=True):
    """
    Simulates each round of the tournament.

    Parameters:
    - round_slots: DataFrame containing information on who is playing in each round.
    - seeds (dict): Dictionary mapping seed values to team IDs.
    - inverted_seeds (dict): Dictionary mapping team IDs to seed values.
    - probas (dict): Dictionary containing matchup probabilities.
    - sim (boolean): Simulates match if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - list: List with winning team IDs for each match.
    - list: List with corresponding slot names for each match.
    """
    winners = []
    slots = []

    for slot, strong, weak in zip(
        round_slots.Slot, round_slots.StrongSeed, round_slots.WeakSeed
    ):
        team_1, team_2 = seeds[strong], seeds[weak]

        # Get the probability of team_1 winning
        proba = probas[str(team_1)][str(team_2)]

        if sim:
            # Randomly determine the winner based on the probability
            winner = np.random.choice([team_1, team_2], p=[proba, 1 - proba])
        else:
            # Determine the winner based on the higher probability
            winner = [team_1, team_2][np.argmax([proba, 1 - proba])]

        # Append the winner and corresponding slot to the lists
        winners.append(winner)
        slots.append(slot)

        seeds[slot] = winner

    # Convert winners to original seeds using the inverted_seeds dictionary
    return [inverted_seeds[w] for w in winners], slots


def run_simulation(brackets=1, seeds=None, preds=None, round_slots=None, sim=True):
    """
    Runs a simulation of bracket tournaments.

    Parameters:
    - brackets (int): Number of brackets to simulate.
    - seeds (pd.DataFrame): DataFrame containing seed information.
    - preds (pd.DataFrame): DataFrame containing prediction information for each match-up.
    - round_slots (pd.DataFrame): DataFrame containing information about the tournament rounds.
    - sim (boolean): Simulates matches if True. Chooses team with higher probability as winner otherwise.

    Returns:
    - pd.DataFrame: DataFrame with simulation results.
    """
    # Get relevant data for the simulation
    seed_dict, inverted_seed_dict, probas_dict = prepare_data(seeds, preds)
    # Lists to store simulation results
    results = []
    bracket = []
    slots = []

    # Iterate through the specified number of brackets
    for b in tqdm(range(1, brackets + 1)):
        # Run single simulation
        r, s = simulate(round_slots, seed_dict, inverted_seed_dict, probas_dict, sim)

        # Update results
        results.extend(r)
        bracket.extend([b] * len(r))
        slots.extend(s)

    # Create final DataFrame
    result_df = pd.DataFrame({"Bracket": bracket, "Slot": slots, "Team": results})

    return result_df


n_brackets = 1
result_m = run_simulation(
    brackets=n_brackets, seeds=seeds_m, preds=preds, round_slots=round_slots, sim=False
)
result_m["Tournament"] = "M"
result_w = run_simulation(
    brackets=n_brackets, seeds=seeds_w, preds=preds, round_slots=round_slots, sim=False
)
result_w["Tournament"] = "W"
submission = pd.concat([result_m, result_w])
submission = submission.reset_index(drop=True)
submission.index.names = ["RowId"]
submission = submission.reset_index()

100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 2732.45it/s]


In [276]:
ss = pd.read_csv(DATA_PATH + "sample_submission.csv")
submission[ss.columns] = submission[ss.columns]
submission[ss.columns].to_csv("Bracket_Prediction.csv", index=False)

# Add team names to csv file

In [277]:
Bracket_Prediction_With_Names_2024 = submission.rename(columns={"Team": "Seed"}).merge(seeds, on=["Seed", "Tournament"], how="left")

teams = pd.concat(
    [
        pd.read_csv(DATA_PATH + "MTeams.csv").assign(Tournament="M"),
        pd.read_csv(DATA_PATH + "WTeams.csv").assign(Tournament="W"),
    ]
)

Bracket_Prediction_With_Names_2024 = Bracket_Prediction_With_Names_2024.merge(teams[["Tournament", "TeamID", "TeamName"]], how="left")

In [278]:
Bracket_Prediction_With_Names_2024.to_csv("Bracket_Prediction_With_Names_2024.csv")

In [281]:
Bracket_Prediction_With_Names_2024.sample(10)

,RowId,Bracket,Slot,Seed,Tournament,TeamID,TeamName
58,58,1,R4Y1,Y01,M,1222,Houston
103,103,1,R2Y1,Y01,W,3231,Indiana
95,95,1,R2W1,W08,W,3378,South Florida
66,66,1,R1W4,W04,W,3417,UCLA
112,112,1,R3W2,W02,W,3268,Maryland
56,56,1,R4W1,W02,M,1266,Marquette
114,114,1,R3X2,X02,W,3234,Iowa
91,91,1,R1Z5,Z05,W,3235,Iowa St
54,54,1,R3Z1,Z01,M,1242,Kansas
120,120,1,R4X1,X01,W,3390,Stanford
